<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [14]</a>'.</span>

# import_to_desktop
Imports the spoke-genelab data and metadata csv files into a Neo4j Desktop database.

Start the `spoke-genelab` Graph DBMS before you run this notebook!

## Setup

In [1]:
import os
import time
import pandas as pd
import seaborn as sns
from py2neo import Graph
import shutil
import requests
from dotenv import load_dotenv
load_dotenv("../.env", override=True)

pd.set_option('display.max_colwidth', None)

In [2]:
def download_http(url, filename):
    if os.path.exists(filename):
        os.remove(filename)
    data = requests.get(url)
    with open(filename, 'wb')as file:
        file.write(data.content)

In [3]:
# copy required files (temporary solution)
download_http("https://raw.githubusercontent.com/pwrose/neo4j-ipycytoscape/master/notebooks/neo4j_utils.py", "neo4j_utils.py")
download_http("https://raw.githubusercontent.com/sbl-sdsc/kg-import/master/notebooks/neo4j_bulk_importer.py", "neo4j_bulk_importer.py")
download_http("https://raw.githubusercontent.com/sbl-sdsc/kg-import/master/notebooks/utils.py", "utils.py")
download_http("https://raw.githubusercontent.com/sbl-sdsc/kg-import/master/notebooks/PrepareNeo4jBulkImport.ipynb", "PrepareNeo4jBulkImport.ipynb")

In [4]:
import neo4j_utils
import neo4j_bulk_importer

### Import the Knowledge Graph
CSV data and metadata files are uploaded into the Neo4j Graph database from the [kg](https://github.com/BaranziniLab/spoke_genelab/tree/main/kg) directory using the [kg-import](https://github.com/sbl-sdsc/kg-import) bulk upload scripts. For a description of the data organization and the specification of metadata [see](https://github.com/sbl-sdsc/kg-import/blob/main/README.md).

In [5]:
neo4j_bulk_importer.import_from_csv_to_neo4j_desktop(verbose=True)

drop_database: '/Users/Peter/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-3d4b95d1-0219-480b-a3c4-ee5a409cc383/bin/cypher-shell' -d system -u neo4j -p neo4jdemo 'DROP DATABASE `spoke-genelab-v0.0.2` IF EXISTS;'


Executing:   0%|          | 0/85 [00:00<?, ?cell/s]

run_bulk_import: cd '/Users/Peter/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-3d4b95d1-0219-480b-a3c4-ee5a409cc383/import'; '/Users/Peter/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-3d4b95d1-0219-480b-a3c4-ee5a409cc383/bin/neo4j-admin' database import full spoke-genelab-v0.0.2 --overwrite-destination --skip-bad-relationships --skip-duplicate-nodes --multiline-fields --array-delimiter='|' @args.txt


Neo4j version: 5.12.0
Importing the contents of these files into /Users/Peter/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-3d4b95d1-0219-480b-a3c4-ee5a409cc383/data/databases/spoke-genelab-v0.0.2:
Nodes:
  [Assay]:
  /Users/Peter/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-3d4b95d1-0219-480b-a3c4-ee5a409cc383/import/header_Assay_n.csv
  /Users/Peter/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-3d4b95d1-0219-480b-a3c4-ee5a409cc383/import/Assay_2024-21-03_n.csv

  [Study]:
  /Users/Peter/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-3d4b95d1-0219-480b-a3c4-ee5a409cc383/import/header_Study_n.csv
  /Users/Peter/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-3d4b95d1-0219-480b-a3c4-ee5a409cc383/import/Study_2024-21-03_n.csv

  [Gene]:
  /Users/Peter/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-3d4b95d1-0

create_database: '/Users/Peter/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-3d4b95d1-0219-480b-a3c4-ee5a409cc383/bin/cypher-shell' -d system -u neo4j -p neo4jdemo 'CREATE OR REPLACE DATABASE `spoke-genelab-v0.0.2`;'


add_indices: '/Users/Peter/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-3d4b95d1-0219-480b-a3c4-ee5a409cc383/bin/cypher-shell' -d spoke-genelab-v0.0.2 -u neo4j -p neo4jdemo -f '/Users/Peter/Library/Application Support/Neo4j Desktop/Application/relate-data/dbmss/dbms-3d4b95d1-0219-480b-a3c4-ee5a409cc383/import/indices.cypher'


### Connect to the local Neo4j Graph database

In [6]:
database = os.environ.get("NEO4J_DATABASE")
username = os.environ.get("NEO4J_USERNAME")
password = os.environ.get("NEO4J_PASSWORD")
stylesheet = os.environ.get("NEO4J_STYLESHEET")

graph = Graph("bolt://localhost:7687", name=database, user=username, password=password)

## Metadata <a class="anchor" id="Metadata"></a>

### Node metadata
spoke_genelab KG is a self-describing KG. The MetaNodes and MetaRelationships define the structure of the KG and the properties of nodes and relationships. The query below lists the nodes and their properties.

In [7]:
query = """
MATCH (n:MetaNode) RETURN n;
"""
df = graph.run(query).to_data_frame()
metadata = df["n"].tolist()
metadata = pd.DataFrame(metadata)
metadata.fillna("", inplace=True)
metadata

,nodeName,identifier,factors_2,material_2,factors_1,material_1,name,material_id_1,material_id_2,technology,measurement,organism,taxonomy,strain,duration,duration_unit
0,Anatomy,UBERON Ontology ID (string),,,,,,,,,,,,,,
1,Assay,GeneLab Data System GLDS-ID-MD5_hashcode(factors and materials) (string),Second factor(s) in transcriptional profiling (string[]),"Second material studied, e.g., cell type, tissue, organ (string)",First factor(s) in transcriptional profiling (string[]),"First material studied, e.g., cell type, tissue, organ (string)",Assay name (string),First material identifier (string),Second material identifier (string),Assay technology (string),Assay measurement type (string),,,,,
2,CellType,Cell Ontology ID (string),,,,,,,,,,,,,,
3,Gene,Gene ENTREZID (string),,,,,,,,,,,,,,
4,MGene,Gene ENTREZID (string),,,,,Gene Symbol (string),,,,,NCBI scientifc organism name (string),NCBI taxonomy id (string),,,
5,Study,NASA Open Science Data Repository ID (string),,,,,Name of the study (string),,,,,NCBI scientific organism name (string),NCBI taxonomy ID (string),Organism strain (string),Exposure duration (string),The unit of the exposure duration (string)


### Metagraph <a class="anchor" id="Metagraph"></a>
The metagraph shows the node labels and relationship types of the KG. Click on a node to display the node metadata.

In [8]:
query = """
MATCH p=(:MetaNode)-->(:MetaNode) RETURN p
"""
subgraph1 = graph.run(query).to_subgraph()

In [9]:
widget1 = neo4j_utils.draw_graph(subgraph1, stylesheet)
widget1.layout.height = "512px"
widget1.set_layout(name="cola", padding=00, nodeSpacing=65, nodeDimensionsIncludeLabels=True, unconstrIter=15000)
widget1

CytoscapeWidget(cytoscape_layout={'name': 'cola', 'padding': 0, 'nodeSpacing': 65, 'nodeDimensionsIncludeLabel…

### Number of Nodes

In [10]:
query = """
MATCH (n) RETURN COUNT(n);
"""
print(f"Total number of nodes: {graph.evaluate(query)}")

Total number of nodes: 127540


In [11]:
query = """
MATCH (n) RETURN labels(n)[0] AS Node, COUNT(n) AS Count
ORDER BY Count DESC
"""
graph.run(query).to_data_frame()

,Node,Count
0,MGene,105984
1,Gene,19755
2,Assay,1698
3,Study,69
4,Anatomy,25
5,MetaNode,6
6,CellType,3


### Number of relationships by relationship type

In [12]:
query = """
MATCH ()-[r]-() RETURN DISTINCT TYPE(r) AS Relationship, COUNT(DISTINCT r) AS Count
ORDER BY Count DESC
"""
graph.run(query).to_data_frame()

,Relationship,Count
0,MEASURED_AmMG,5046101
1,IS_ORTHOLOG_MGiG,60972
2,PERFORMED_SpA,1698
3,INVESTIGATED_AiA,1665
4,INVESTIGATED_AiCT,55
5,MetaRelationship,5


### Example query

In [13]:
tissue = "Liver"
threshold = -3.0
query = """
MATCH p=(:Mission)--(s:Study)-[d:DOWNREGULATION_SdMG]->(:MouseGene) WHERE s.tissue = $tissue AND d.log2fc < $threshold RETURN p
"""
subgraph2 = graph.run(query, tissue=tissue, threshold=threshold).to_subgraph()

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [14]:
widget2 = neo4j_utils.draw_graph(subgraph2, stylesheet)
widget2.layout.height = "512px"
widget2.set_layout(name="cola", padding=0, nodeSpacing=65, nodeDimensionsIncludeLabels=True, unconstrIter=15000)
widget2

AttributeError: 'NoneType' object has no attribute 'nodes'

## Fulltext Search <a class="anchor" id="Fulltext Search"></a>

### Full text query by keyword or phrase
A full text query returns all nodes that match the text query ([Query Syntax](https://lucene.apache.org/core/5_5_0/queryparser/org/apache/lucene/queryparser/classic/package-summary.html#Overview)). For exact matches, enclose the phrase in double quotes, e.g., ```'"muscle"'```.

[Learn more about full-text searches.](https://graphaware.com/neo4j/2019/01/11/neo4j-full-text-search-deep-dive.html)

In [ ]:
phrase = '"muscle"'
top_n = 5 # show top n hits

In [ ]:
query = """
CALL db.index.fulltext.queryNodes("fulltext", $phrase) YIELD node, score
RETURN node.identifier AS identifier, LABELS(node)[0] AS type, node.tissue AS tissue
"""
graph.run(query, phrase=phrase).to_data_frame().head(top_n)

### Full text query using boolean operators
The full text query supports a variety of query types, including fuzzy, proximity, and range queries, as well as boolean operators ([Query Syntax](https://lucene.apache.org/core/5_5_0/queryparser/org/apache/lucene/queryparser/classic/package-summary.html#Overview)). The following example uses a query with an ```AND``` operator.

In [ ]:
phrase = 'Rodent Research AND Mus musculus'

In [ ]:
query = """
CALL db.index.fulltext.queryNodes("fulltext", $phrase) YIELD node, score
RETURN node.identifier AS identifier, LABELS(node)[0] AS type, node.name AS name, score
ORDER BY type
"""
graph.run(query, phrase=phrase).to_data_frame()